# 通过微调进行迁移学习

`tensorflow`中没有预训练好的模型, 但在`tf-slim`中包含了很多著名网络的网络结构图和预训练模型, 比如我们前面介绍过的`alexnet`,`vgg`,`inception`,`resnet`以及它们的一些变体. 在使用的时候我们先用`tf-slim`定义好的网络构建计算图, 然后再加载预训练模型的参数就行了, 非常简单.

下面我们用一个例子来演示一些微调

In [ ]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import tensorflow as tf
import tensorflow.contrib.slim as slim
from utils.custom_input import read
from utils.learning import train_with_bn

我们先来看一下数据集

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
root_path = './hymenoptera_data/train/'
im_list = [os.path.join(root_path, 'ants', i) for i in os.listdir(root_path + 'ants')[:4]]
im_list += [os.path.join(root_path, 'bees', i) for i in os.listdir(root_path + 'bees')[:5]]

nrows = 3
ncols = 3
figsize = (8, 8)
_, figs = plt.subplots(nrows, ncols, figsize=figsize)
for i in range(nrows):
    for j in range(ncols):
        figs[i][j].imshow(Image.open(im_list[nrows*i+j]))
        figs[i][j].axes.get_xaxis().set_visible(False)
        figs[i][j].axes.get_yaxis().set_visible(False)
plt.show()

这里我们用封装在`custom_input.py`里的函数读取数据

In [ ]:
category_label_dict, train_names, train_imgs, train_labels, train_examples = read('hymenoptera_data/train/', shuffle=True, batch_size=64)
_, val_names, val_imgs, val_labels, val_examples = read('hymenoptera_data/val/', category_label_dict, train=False, batch_size=128)

## Fine-tune

使用`tf-slim`中的网络定义模型

In [ ]:
import tensorflow.contrib.slim.python.slim.nets.resnet_v2 as resnet

In [ ]:
def slim_resnet(inputs, num_classes=2, is_training=True, scope='slim_resnet', reuse=None):
    logits, endpts = resnet.resnet_v2_50(inputs, 10, is_training, reuse=reuse, scope=scope)
    out = tf.squeeze(logits, [1, 2])
    
    return out, endpts

In [ ]:
is_training_ph = tf.placeholder(tf.bool, name='is_training')

with slim.arg_scope(resnet.resnet_arg_scope(weight_decay=0.0005)):
    train_out, train_endpts = slim_resnet(train_imgs, 2, is_training_ph)
    val_out, val_endpts = slim_resnet(val_imgs, 2, is_training_ph, reuse=True)

In [ ]:
with tf.variable_scope('loss'):
    train_loss = tf.losses.sparse_softmax_cross_entropy(labels=train_labels, logits=train_out, scope='train')
    val_loss = tf.losses.sparse_softmax_cross_entropy(labels=val_labels, logits=val_out, scope='val')

In [ ]:
with tf.name_scope('accuracy'):
    with tf.name_scope('train'):
        train_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(train_out, axis=-1, output_type=tf.int32), train_labels), tf.float32))
    with tf.name_scope('val'):
        val_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(val_out, axis=-1, output_type=tf.int32), val_labels), tf.float32))

In [ ]:
lr = 0.01

opt = tf.train.MomentumOptimizer(lr, momentum=0.9)

In [ ]:
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = opt.minimize(train_loss)

### 恢复预训练模型参数

- **Point1**

首先我们注意到模型文件是`.ckpt`文件, 里面存储的模型参数都有自己的参数名称, 在我们恢复参数数值的过程中, `tensorflow`会按照参数名称进行加载. 

通常预训练模型的参数名和我们自己定义的参数名不完全相同, 但存在对应关系. 比如说`tf-slim`的`resnet50`模型的所有参数名称都有`'resnet_v2_50'`前缀, 而我们所有参数模型的前缀是`'slim_resnet'`.

那么我们建立一个字典, 预训练模型的参数名称作为`key`, 参数作为`value`. 这样在恢复参数的过程中, `tensorflow`根据字典的`key`找到预训练模型中的参数数值, 根据`value`找到需要被恢复的参数变量, 这样就完成了参数恢复.

- - -

- **Point2**

然后需要注意我们能够恢复哪些参数, 例如在这里最后一个分类层的参数, `imagenet`比赛中是`1000`分类, 所以它的最后一层是$2048\times1000$的权重, 而在这个2分类问题中, 我们的参数是$2048\times2$的, 所以不能进行恢复, 只能初始化再训练

In [ ]:
pretrained_model_path = 'pretrained_models/resnet_v2_50/model.ckpt'

# 构建需要恢复的名称变量对应字典
vars_to_restore = {}

for var in tf.global_variables():
    # 需要恢复的模型变量
    if var in tf.model_variables():
        
        # 不需要恢复最后一层的参数
        if 'logit' not in var.op.name:
            
            # 找到预训练模型中参数变量的名字
            pretrained_model_var_name = var.op.name.replace('slim_resnet', 'resnet_v2_50')
            
            # 添加到对应字典中
            vars_to_restore[pretrained_model_var_name] = var

vars_to_init = list(filter(lambda var: var not in vars_to_restore.values(), tf.global_variables()))

现在我们来看看`vars_to_restore`和`vars_to_init`里面的元素

In [ ]:
print('vars_to_restore')
print('-'*30)
for i in range(10):
    print(list(vars_to_restore.values())[i].name)
print('-'*30)
print('vars_to_init')
print('-'*30)
for i in range(10):
    print(vars_to_init[i].name)

可以发现, `bn`层的`moving_mean`和`moving_average`都是需要恢复的

最后一个分类层(`logits`)和动量值是需要被训练的

In [ ]:
sess = tf.Session()

### 建立恢复变量的读取器

In [ ]:
restorer = tf.train.Saver(vars_to_restore)

### 恢复模型变量

In [ ]:
restorer.restore(save_path=pretrained_model_path, sess=sess)

那么现在我们来用`graph`的`get_tensor_by_name`来看看恢复后参数的数值

In [ ]:
print(sess.run(tf.get_default_graph().get_tensor_by_name('slim_resnet/conv1/weights:0')))

### 初始化其他变量

In [ ]:
sess.run(tf.variables_initializer(vars_to_init))

开始训练

In [ ]:
_ = train_with_bn(sess, 
                              train_op, 
                              train_loss, 
                              train_acc, 
                              val_loss, 
                              val_acc, 
                              150, 
                              is_training_ph, 
                              train_examples=train_examples, 
                              val_examples=val_examples, 
                              train_batch=64, 
                              val_batch=128, 
                              train_log_step=10, 
                              val_log_step=100)

可以看到, 最后验证集上达到了0.89的正确率

In [ ]:
sess.close()

恢复模型后还可以仅仅训练最后一个分类层的参数而保持其他模型参数不变, 这只需要构建训练过程中`minimize`函数的帮助就可以实现

## Fix Paramters

In [ ]:
vars_to_train = list(filter(lambda var: var not in vars_to_restore.values(), tf.trainable_variables()))

In [ ]:
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.control_dependencies(update_ops):
    train_op = opt.minimize(train_loss, var_list=vars_to_train)

In [ ]:
vars_to_init = list(filter(lambda var: var not in vars_to_restore.values(), tf.global_variables()))

重新加载模型变量

In [ ]:
sess = tf.Session()
restorer.restore(sess, pretrained_model_path)
sess.run(tf.variables_initializer(vars_to_init))

重新训练

In [ ]:
_ = train_with_bn(sess, 
                              train_op, 
                              train_loss, 
                              train_acc, 
                              val_loss, 
                              val_acc, 
                              150, 
                              is_training_ph, 
                              train_examples=train_examples, 
                              val_examples=val_examples, 
                              train_batch=64, 
                              val_batch=128, 
                              train_log_step=10, 
                              val_log_step=100)

再来看看参数值是否变化

In [ ]:
print(sess.run(tf.get_default_graph().get_tensor_by_name('slim_resnet/conv1/weights:0')))

这样其实相当于把前面的卷积网络当作特征提取器, 然后用一个单层神经网络去训练

In [ ]:
sess.close()

作为对比, 我们再来看看完全不使用预训练模型时的效果

In [ ]:
with tf.control_dependencies(update_ops):
    train_op = opt.minimize(train_loss)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
_ = train_with_bn(sess, 
                              train_op, 
                              train_loss, 
                              train_acc, 
                              val_loss, 
                              val_acc, 
                              150, 
                              is_training_ph, 
                              train_examples=train_examples, 
                              val_examples=val_examples, 
                              train_batch=64, 
                              val_batch=128, 
                              train_log_step=10, 
                              val_log_step=100)

通过上面的结果可以看到，使用预训练的模型能够非常快的达到 90% 左右的验证集准确率，而不使用预训练模型训练集在相同步长内都难以收敛，所以使用一个预训练的模型能够在较小的数据集上也取得一个非常好的效果，因为对于图片识别分类任务，最底层的卷积层识别的都是一些通用的特征，比如形状、纹理等等，所以对于很多图像分类、识别任务，都可以使用预训练的网络得到更好的结果。